<a href="https://colab.research.google.com/github/CYBERIC-EA/ANIME-RECOMMENDER/blob/main/ANIME_RECOMMENDER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [241]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TABLE OF CONTENTS
1. Introduction
2. Data Id
3. Libraries
4. Preprocessing and Data Analysis
5. Cosine Similarity Model
6. Conclusion

This data set contains information on user preference data from 73,516 users on 16,621 anime. Each user is able to add anime to their completed list and give it a rating and this data set is a compilation of those ratings. The data was scraped thanks to myanimelist.net API.

SECTION 2
DATA INDENTIFICATION

In [242]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter
import scipy as sp #pivot egineering


#ML model
from sklearn.metrics.pairwise import cosine_similarity


#default theme and settings
pd.options.display.max_columns

#warning hadle

warnings.filterwarnings("always")
warnings.filterwarnings("ignore")


In [243]:
for dirname, _, filenames in os.walk('recommend/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [244]:
ratingPath = 'recommend/rating_complete.csv'
animePath = 'recommend/anime.csv'

In [245]:
anime_df = pd.read_csv('/content/drive/MyDrive/Data Science/anime.csv')
anime_df.head()

,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


Create a DataFrame for your ratings

In [246]:
rating_df = pd.read_csv('/content/drive/MyDrive/Data Science/rating_complete.csv')
rating_df.head()

,user_id,anime_id,rating
0,0,147,5.0
1,1,1512,4.5
2,1,599,4.0
3,1,2292,4.5
4,1,1078,3.0


Data Shapes of the dataframe

In [247]:
print('anime_df')
anime_df.shape


anime_df


(16621, 17)

In [248]:
print('rating_df')
rating_df.shape

rating_df


(8765945, 3)

Info of the dataframe

In [249]:
print("Anime:\n")
print(anime_df.info())

Anime:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16621 entries, 0 to 16620
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Anime-PlanetID    16621 non-null  int64 
 1   Name              16621 non-null  object
 2   Alternative Name  16621 non-null  object
 3   Rating Score      16621 non-null  object
 4   Number Votes      16621 non-null  object
 5   Tags              16621 non-null  object
 6   Content Warning   16621 non-null  object
 7   Type              16621 non-null  object
 8   Episodes          16621 non-null  object
 9   Finished          16621 non-null  bool  
 10  Duration          16621 non-null  object
 11  StartYear         16621 non-null  object
 12  EndYear           16621 non-null  object
 13  Season            16621 non-null  object
 14  Studios           16621 non-null  object
 15  Synopsis          16615 non-null  object
 16  Url               16621 non-null  object
dtypes: b

In [250]:
print("Rating:\n")
print(rating_df.info())

Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8765945 entries, 0 to 8765944
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   int64  
 1   anime_id  int64  
 2   rating    float64
dtypes: float64(1), int64(2)
memory usage: 200.6 MB
None


HANDLING MISSING DATA FIELDS

In [251]:
print("Missing Values in the Anime Dataset (%) : \n\n")
print(round(anime_df.isnull().sum().sort_values(ascending=False)/len(anime_df.index), 4 ) * 100)

#some missing values are given as unknown.
#if you really wanna know how mmany unkown values there are yk what to do.



print('\n', '=+=' * 36 , '\n')

print("Missing Values in the Ratings Dataset (%) : \n\n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df.index), 4 ) * 100)


Missing Values in the Anime Dataset (%) : 


Synopsis            0.04
Anime-PlanetID      0.00
Finished            0.00
Studios             0.00
Season              0.00
EndYear             0.00
StartYear           0.00
Duration            0.00
Episodes            0.00
Name                0.00
Type                0.00
Content Warning     0.00
Tags                0.00
Number Votes        0.00
Rating Score        0.00
Alternative Name    0.00
Url                 0.00
dtype: float64

 =+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+==+= 

Missing Values in the Ratings Dataset (%) : 


user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [252]:

print(anime_df['Type'].mode())
print(anime_df['Tags'].mode())
print(anime_df['Rating Score'].mode())


0    TV
dtype: object
0    Unknown
dtype: object
0    Unknown
dtype: object


The most frequent type is TV
The most frequent tags are unkown because a video may have multiple tags this may result in a problem


In [253]:

# filling mode value for tags and type
anime_df['Tags'] = anime_df['Tags'].apply(lambda x: np.nan if x=='Unknown' else x)
anime_df['Tags'] = anime_df['Tags'].fillna(anime_df['Tags'].dropna().mode().values[0])


anime_df['Type'] = anime_df['Type'].apply(lambda x: np.nan if x=='Unknown' else x)
anime_df['Type'] = anime_df['Type'].fillna(anime_df['Type'].dropna().mode().values[0])

# deleting anime with 0 rating

anime_df['Rating Score'] = anime_df['Rating Score'].apply(lambda x: 0 if x=='Unknown' else x)
#anime_df['Rating Score'] = anime_df['Rating Score'].fillna(anime_df['Rating Score'].dropna().mode().values[0])
anime_df['Rating Score'] = anime_df['Rating Score'].astype('float')
anime_df=anime_df[~np.isnan(anime_df["Rating Score"])]


#checking if all null values are filled
anime_df.isnull().sum()

Anime-PlanetID      0
Name                0
Alternative Name    0
Rating Score        0
Number Votes        0
Tags                0
Content Warning     0
Type                0
Episodes            0
Finished            0
Duration            0
StartYear           0
EndYear             0
Season              0
Studios             0
Synopsis            6
Url                 0
dtype: int64

In [254]:

# making sure fields that are empty will show up as noN
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head()

,user_id,anime_id,rating
0,0,147,5.0
1,1,1512,4.5
2,1,599,4.0
3,1,2292,4.5
4,1,1078,3.0



Now we will engineer our Dataframe in the following steps:
We want to recommend anime series only so the the relevant type is TV
We make a new Dataframe combining both anime and rating on the anime_id column.
Leaving only user_id, name and rating as the Df.
For computing purpose only we compute our Df based only on the first 7500 users.


In [255]:
#step 1
anime_df = anime_df[anime_df['Type']=='TV']

#step 2
rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'Anime-PlanetID', suffixes= ['_user', ''])

#step 3 - renaming rated_anime
rated_anime =rated_anime[['user_id', 'Name', 'Rating Score', 'Tags', 'Url']]

#step 4
rated_anime_7500= rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()

,user_id,Name,Rating Score,Tags,Url
0,0,Fruits Basket,4.139,"Comedy, Drama, Fantasy, Romance, Shoujo, Anima...",https://www.anime-planet.com/anime/fruits-basket
1,2,Fruits Basket,4.139,"Comedy, Drama, Fantasy, Romance, Shoujo, Anima...",https://www.anime-planet.com/anime/fruits-basket
2,6,Fruits Basket,4.139,"Comedy, Drama, Fantasy, Romance, Shoujo, Anima...",https://www.anime-planet.com/anime/fruits-basket
3,7,Fruits Basket,4.139,"Comedy, Drama, Fantasy, Romance, Shoujo, Anima...",https://www.anime-planet.com/anime/fruits-basket
4,23,Fruits Basket,4.139,"Comedy, Drama, Fantasy, Romance, Shoujo, Anima...",https://www.anime-planet.com/anime/fruits-basket


TIME FOR THE FUN STUFF!!


Pivot Table for similarity
We will create a pivot table of users as rows and tv show names as columns. The pivot table will help us will be analized for the calcuations of similarity.

In [256]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns = ['Name'], values = 'Rating Score')

#Tagpivot = rated_anime_7500.pivot_table(index=['Tags'], columns = ['Name'], values = 'Rating Score')
Tagpivot = rated_anime_7500.pivot_table(index=['Name'], columns = ['Tags'], values = 'Rating Score')
#pivot.head()

A pivot table is a table of grouped values that aggregates the individual items of a more
extensive table within one or more discrete categories

Now we will engineer our pivot table in the following steps:
Value normalization.
Filling Nan values as 0.
Transposing the pivot for the next step.
Dropping columns with the values of 0 (unrated).
Using scipy package to convert to sparse matrix format for the similarity computation.

In [257]:
# step 1
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
pivot_t = Tagpivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# step 2
pivot_n.fillna(0, inplace=True)
pivot_t.fillna(0,inplace=True)

# step 3
pivot_n = pivot_n.T
pivot_t = pivot_t.T

# step 4
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]
pivot_t = pivot_t.loc[:, (pivot_t != 0).any(axis=0)]

# step 5
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)
pivot_t_sparce = sp.sparse.csr_matrix(pivot_t.values)


In [258]:
#model based on anime similarity
anime_similarity = cosine_similarity(piv_sparse)
tag_similarity = cosine_similarity(pivot_t_sparce)


#Df of anime similarities
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)
tag_sim_df = pd.DataFrame(tag_similarity, index = pivot_t.index, columns = pivot_t.index)


Done with Pivot, sparsing and cosine_similarity

Next build the function for the anime recommender


In [259]:
def anime_recommendation(ani_name):
    """
    This function will return the top 5 shows with the highest cosine similarity value and show match percent
    
    example:
    >>>Input: 
    
    anime_recommendation('Death Note')
    
    >>>Output: 
    
    Recommended because you watched Death Note:

                    #1: Code Geass: Hangyaku no Lelouch, 57.35% match
                    #2: Code Geass: Hangyaku no Lelouch R2, 54.81% match
                    #3: Fullmetal Alchemist, 51.07% match
                    #4: Shingeki no Kyojin, 48.68% match
                    #5: Fullmetal Alchemist: Brotherhood, 45.99% match 

               
    """
    
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1

In [260]:
anime_recommendation('Fruits Basket')

Recommended because you watched Fruits Basket:

#1: Ouran High School Host Club, 37.18% match
#2: Cardcaptor Sakura, 33.83% match
#3: Full Metal Panic!, 33.61% match
#4: The Melancholy of Haruhi Suzumiya, 33.58% match
#5: Black Butler, 33.45% match


In [261]:
def genre_recommendation(genre_name):
    number = 1
    print('Recommended because you watched {}:\n'.format(genre_name))
    
    #anime_df = anime_df[anime_df['Type']=='TV']
    

    seen = set()
    
    #for i in range(len(rated_anime['Name'])):
    sui = rated_anime.sort_values(by = 'Rating Score', ascending = False)
    res =  list()
    for i in range(len(rated_anime)):
          if genre_name in sui['Tags'][i]:          
              
              if sui['Name'][i] in seen:
                  continue
              else:
                  seen.add(sui['Name'][i])
              res.append((sui['Name'][i], sui['Url'][i]))

    ffs = res[1: 5]
    result = [i for i in ffs]
    [print(i[0], ':', i[1]) for i in result]



In [262]:
genre_recommendation('Action')


Recommended because you watched Action:

Mobile Fighter G Gundam : https://www.anime-planet.com/anime/mobile-fighter-g-gundam
After War Gundam X : https://www.anime-planet.com/anime/after-war-gundam-x
Digimon Season 2: Digital Monsters : https://www.anime-planet.com/anime/digimon-season-2-digital-monsters
A Certain Scientific Railgun : https://www.anime-planet.com/anime/a-certain-scientific-railgun


In [263]:
def popularity():
  sui = rated_anime.sort_values(by = 'Rating Score', ascending = False)
  
  res = list()
  seen = set()
  for i in range(len(rated_anime)):
    if sui['Name'][i] in seen:
      continue
    else:
      seen.add(sui['Name'][i])
      res.append((sui['Name'][i], sui['Url'][i]))
      
      
      

  ffs = res[1: 5]
  result = [i for i in ffs]
  [print(i[0],':',  i[1]) for i in result]



In [264]:
popularity()

Clannad : https://www.anime-planet.com/anime/clannad
Mobile Suit Gundam : https://www.anime-planet.com/anime/mobile-suit-gundam
Tengen Toppa Gurren Lagann : https://www.anime-planet.com/anime/tengen-toppa-gurren-lagann
Mobile Fighter G Gundam : https://www.anime-planet.com/anime/mobile-fighter-g-gundam
